#Workshop 7

* Mateo Cano Solís - 1037660293
* Keyla García Jaimes - 1026159841

In [5]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import pathlib

from tensorflow.keras import layers
from tensorflow.keras import losses

##Descargar y explorar Stack Overflow dataset
Descarguemos y extraigamos el conjunto de datos, luego exploremos la estructura del directorio.

In [6]:
# Downloading and exploring the StackOverflow dataset
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'
dataset = tf.keras.utils.get_file(
  'stack_overflow_16k.tar.gz',
  data_url,
  untar=True,  
  cache_dir='stack_overflow',
  cache_subdir='.')
dataset_dir = pathlib.Path(dataset).parent

Imprimamos un archivo de ejemplo

In [7]:
train_dir = dataset_dir/'train'
sample_file = os.path.join(train_dir, 'python/1755.txt')
with open(sample_file) as f:
  print(f.read())

why does this blank program print true x=true.def stupid():.    x=false.stupid().print x



Dentro de la carpeta esta el set de entrenamiento y el de testeo

In [8]:
os.listdir(dataset_dir)

['stack_overflow_16k.tar.gz', 'README.md', 'test', 'train']

Dentro del de entranamiento están los siguientes conjuntos de programas, que permiten etiquetar cada una de las preguntas

In [9]:
os.listdir(train_dir)

['csharp', 'java', 'python', 'javascript']

##Cargar Dataset
No es necesario remover carpetas ya que no se cuenta con carpetas adicionales a las clases necesarias. 
Al ejecutar un experimento de aprendizaje automático, se recomienda dividir el conjunto de datos en tres divisiones: entrenar, [validación](https:/ /developers.google.com/machine-learning/glossary#validation_set) y test. El conjunto de datos de Stack OverFlow ya se ha dividido en entrenamiento y prueba, pero carece de un conjunto de validación. Vamos a crear un conjunto de validación usando una división 80:20 de los datos de entrenamiento usando el argumento validation_split a continuación.

In [10]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(
  train_dir,
  batch_size=batch_size,
  validation_split=0.2,
  subset='training',
  seed=seed)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


Hay 8000 ejemplos en la carpeta de capacitación, de los cuales usará el 80 % (o 6400) para training.
A continuación, se creará un conjunto de datos de validación y prueba. Utilizará las 1600 revisiones restantes del conjunto de capacitación para la validación

In [11]:
raw_val_ds = tf.keras.utils.text_dataset_from_directory(
    train_dir, 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [12]:
raw_test_ds = tf.keras.utils.text_dataset_from_directory(
    train_dir, 
    batch_size=batch_size)

Found 8000 files belonging to 4 classes.


In [13]:
for i, label in enumerate(raw_train_ds.class_names):
  print("Label", i, "corresponds to", label)

Label 0 corresponds to csharp
Label 1 corresponds to java
Label 2 corresponds to javascript
Label 3 corresponds to python


### Preparar el dataset para el entrenamiento
* Estandarización
* Tokenización
* Vectorización

In [14]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return lowercase

In [15]:
max_features = 10000
sequence_length = 250

vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

A continuación, llamará a `adapt` para ajustar el estado de la capa de preprocesamiento al conjunto de datos. Esto hará que el modelo genere un índice de cadenas a números enteros.

In [16]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

Vamos a crear una función para ver el resultado de usar esta capa para preprocesar algunos datos.

In [17]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [18]:
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))

Review tf.Tensor(b'"unit testing of setters and getters teacher wanted us to do a comprehensive unit test. for me, this will be the first time that i use junit. i am confused about testing set and get methods. do you think should i test them? if the answer is yes; is this code enough for testing?..  public void testsetandget(){.    int a = 10;.    class firstclass = new class();.    firstclass.setvalue(10);.    int value = firstclass.getvalue();.    assert.asserttrue(""error"", value==a);.  }...in my code, i think if there is an error, we can\'t know that the error is deriving because of setter or getter."\n', shape=(), dtype=string)
Label java
Vectorized review (<tf.Tensor: shape=(1, 250), dtype=int64, numpy=
array([[   1,  908,   11, 3870,   10, 2472, 3454,  708, 1636,    4,   43,
           6,    1, 1395, 2618,   13, 1538,   16,   69,   32,    2,  105,
         127,   15,    5,   70,    1,    5,   33, 1061,  193,  908,  156,
          10,   48, 1912,   43,   65,  301,   92,    5,  2

Veamos la representación numerica que palabras posee


In [19]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

1287 --->  direction
 313 --->  instead
Vocabulary size: 10000


## Se aplicará la capa TextVectorization al conjunto de datos de entrenamiento, validación y prueba.

In [20]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [21]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Crear el modelo 

In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [23]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [24]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [25]:
vocab_size = 20000 
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 250


inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(4, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

Para la 3era epoca el modelo alcanzo un acuracy mayor al 80% además de no presentar overfitting, para mas epocas el modelo comienza a aprenderse los ejemplos

In [26]:
model.compile(
    optimizer="adam", loss= losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy']
)
history = model.fit(
    train_ds, batch_size=32, epochs=5, validation_data=val_ds
)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


200/200 [==============================] - 39s 185ms/step - loss: 1.3046 - accuracy: 0.3483 - val_loss: 1.0961 - val_accuracy: 0.5344
Epoch 2/5
200/200 [==============================] - 35s 174ms/step - loss: 0.8371 - accuracy: 0.6519 - val_loss: 0.5376 - val_accuracy: 0.8138
Epoch 3/5
200/200 [==============================] - 36s 179ms/step - loss: 0.4165 - accuracy: 0.8413 - val_loss: 0.5270 - val_accuracy: 0.8112
Epoch 4/5
200/200 [==============================] - 34s 172ms/step - loss: 0.2246 - accuracy: 0.9255 - val_loss: 0.7214 - val_accuracy: 0.8056
Epoch 5/5
200/200 [==============================] - 35s 173ms/step - loss: 0.1236 - accuracy: 0.9631 - val_loss: 0.8274 - val_accuracy: 0.7931


Para el set de testeo tenemos un acuracy que llega al 95%, lo que muestra la gran precision del modelo

In [27]:
loss, accuracy = model.evaluate(test_ds,return_dict = True)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

250/250 [==============================] - 20s 81ms/step - loss: 0.2054 - accuracy: 0.9469
Loss:  loss
Accuracy:  accuracy


Por tal motivo se muestra el gran trabajo del transformer para generalizar con problemas multiclases además de brindar accuracy tan altas. También la gran ventaja al ser modelos más livianos y en teoria con una arquitectura mas liviana. 